In [1]:
%%init_spark
launcher.conf.spark.app.name = "archita_spark3"
launcher.conf.spark.local.dir = "/data/tmp/spark"
launcher.conf.spark.driver.memory = "100g"
launcher.conf.spark.executor.memory = "50g"
launcher.conf.spark.executors = "21"
launcher.conf.spark.executor.cores = "21"
launcher.conf.spark.sql.shuffle.partitions = 210
launcher.conf.spark.sql.shuffle.minPartitions = 20
launcher.conf.spark.ui.showConsoleProgress = "true"
launcher.conf.spark.yarn.keytab = "/home/jioapp/aditya/jiomart_cluster/couture.keytab"
launcher.conf.spark.yarn.principal = "couture@COUTURE.AI"
launcher.conf.spark.driver.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"
launcher.conf.spark.executor.extraJavaOptions = "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"

In [2]:
val history = spark.read.parquet("hdfs://10.144.96.170:8020/data1/searchengine/phase2/Dataset/02022025/V1/HistoryInteractionForNDCG")

Intitializing Scala interpreter ...

Spark Web UI available at http://10.166.181.219:4051
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1739277854568)
SparkSession available as 'spark'


history: org.apache.spark.sql.DataFrame = [search_term: string, option_code: string ... 5 more fields]


In [3]:
history.printSchema()

root
 |-- search_term: string (nullable = true)
 |-- option_code: string (nullable = true)
 |-- norm_revr: double (nullable = true)
 |-- norm_ordr_clicks: double (nullable = true)
 |-- norm_ctr: double (nullable = true)
 |-- norm_revr_clicks: double (nullable = true)
 |-- target: double (nullable = true)



In [4]:
history.show()

+--------------------+--------------------+---------+----------------+--------------------+----------------+------------------+
|         search_term|         option_code|norm_revr|norm_ordr_clicks|            norm_ctr|norm_revr_clicks|            target|
+--------------------+--------------------+---------+----------------+--------------------+----------------+------------------+
|               women|2147483647_014ber...|      0.0|             0.0|                 0.2|             0.0| 3.900518181272789|
|               women|2147483647_019nud...|      0.0|             0.0|                 1.0|             0.0| 4.374065485771738|
|               women|2147483647_01blac...|      0.0|             0.0| 0.03719599427753934|             0.0|3.8233650762322484|
|               women|2147483647_01blac...|      0.0|             0.0|0.024774774774774775|             0.0|  3.81774934491952|
|          foundation|2147483647_01bron...|      0.0|             0.0|0.018102824040550327|             

In [5]:
var df_tq = spark.read.parquet("hdfs://10.144.96.170:8020/data1/searchengine/testing/ajio/query_product_scored_top_100")
.withColumn("product",explode(col("products")))
.drop("products")
.select(col("query"),col("product.colorGroup_string").alias("id"),col("product.score").alias("score"))

df_tq: org.apache.spark.sql.DataFrame = [query: string, id: string ... 1 more field]


In [6]:
df_tq.show()

+--------------------+--------------------+------------------+
|               query|                  id|             score|
+--------------------+--------------------+------------------+
|avaasa mix n&#39;...|441144528_olivegreen|0.7518140609999999|
|avaasa mix n&#39;...|      441124685_blue|0.7420315622033999|
|avaasa mix n&#39;...|      441145550_blue|   0.7367807992086|
|avaasa mix n&#39;...|      441146701_teal|   0.7360080622821|
|avaasa mix n&#39;...|     441133434_peach|   0.7273994481226|
|avaasa mix n&#39;...|441149941_pistagreen|0.7266052330579408|
|avaasa mix n&#39;...|  441115970_offwhite|0.7249970551199432|
|avaasa mix n&#39;...|      443007662_pink|0.7234900550381574|
|avaasa mix n&#39;...|     441141761_coral|0.7226483943971731|
|avaasa mix n&#39;...|    443003252_ltpink| 0.721764438020237|
|avaasa mix n&#39;...| 441103249_limegreen|0.7215667576602801|
|avaasa mix n&#39;...|     441124214_black|0.7195380612258001|
|avaasa mix n&#39;...|      441134610_grey|  0.71755374

In [7]:
var joined_df = df_tq.join(history,history("search_term") === df_tq("query") && history("option_code") === df_tq("id"))
.drop("target")
.drop(history("search_term"))
.drop(history("option_code"))

joined_df: org.apache.spark.sql.DataFrame = [query: string, id: string ... 5 more fields]


In [8]:
joined_df.printSchema()

root
 |-- query: string (nullable = true)
 |-- id: string (nullable = true)
 |-- score: double (nullable = true)
 |-- norm_revr: double (nullable = true)
 |-- norm_ordr_clicks: double (nullable = true)
 |-- norm_ctr: double (nullable = true)
 |-- norm_revr_clicks: double (nullable = true)



In [9]:
var sr = joined_df.stat.corr("score","norm_revr")
var so = joined_df.stat.corr("score","norm_ordr_clicks")
var sc = joined_df.stat.corr("score","norm_ctr")

25/02/11 18:17:07 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 18:17:07 WARN SparkConf: The configuration key 'spark.yarn.principal' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.principal' instead.
25/02/11 18:17:08 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 18:17:08 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.keytab' instead.
25/02/11 18:17:08 WARN SparkConf: The configuration key 'spark.yarn.keytab' has been deprecated as of Spark 3.0 and may be removed in the future. Please use the new key 'spark.kerberos.k

sr: Double = 0.008066402610930782
so: Double = 0.010129465574712945
sc: Double = 0.15696886887445796


In [13]:
println(raw"Correlation(score,revr):- $sr")
println(raw"Correlation(score,order_clicks):- $so")
println(raw"Correlation(score,ctr):- $sc")

Correlation(score,revr):- 0.008066402610930782
Correlation(score,order_clicks):- 0.010129465574712945
Correlation(score,ctr):- 0.15696886887445796
